In [1]:
from transformers import AutoTokenizer
from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error()
from transformers.utils import logging as hf_logging
hf_logging.set_verbosity_error()
tok = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_fast=False, src_lang="eng_Latn")

tok.eos_token_id

/home/dev371/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/home/dev371/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38
the `fairseq_tokens_to_ids` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38
Special tokens have been added i

2

In [2]:
#!pip install -q librosa soundfile transformers
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import os, numpy as np, tensorflow as tf, librosa, soundfile as sf
from IPython.display import Audio, display
from transformers import AutoTokenizer

# لاگ‌های اضافی کمتر
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
for gpu in tf.config.list_physical_devices("GPU"):
    try: tf.config.experimental.set_memory_growth(gpu, True)
    except: pass


2025-10-22 20:50:27.526266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761153627.537870  146227 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761153627.541420  146227 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761153627.550866  146227 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761153627.550875  146227 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761153627.550877  146227 computation_placer.cc:177] computation placer alr

In [3]:
from MyTTSModel import TransformerTTS  # همان فایلی که مدل TTS را نوشتی

# پارامترها باید با آموزش هم‌خوانی داشته باشند
SR          = 22050
N_FFT       = 1024
HOP         = 256
WIN         = 1024
N_MELS      = 80
FMIN        = 0.0
FMAX        = 8000.0
PAD_ID      = 1
MAX_SRC_LEN = 256
MAX_MEL_LEN = 2000  # سقف منطقی برای تولید

# توکنایزر NLLB (فعلاً انگلیسی)
tok = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_fast=False, src_lang="eng_Latn")

# ساخت/لود مدل
core = TransformerTTS(
    num_layers=6, d_model=256, num_heads=8, dff=1024,
    input_vocab_size=tok.vocab_size, n_mels=N_MELS,
    dropout_rate=0.1, pad_id=PAD_ID
)

core.build_for_load(max_src_len=MAX_SRC_LEN, max_tgt_len=MAX_MEL_LEN)
# وزن‌های آموزش‌داده‌شده‌ات
os.makedirs("checkpoints", exist_ok=True)
core_path = "checkpoints/tts_core_last.weights.h5"

if os.path.exists(core_path):
    core.load_weights(core_path)
    print("✅ Weights loaded.")
else:
    print("⚠️ Checkpoint not found:", core_path)


/home/dev371/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
I0000 00:00:1761153634.273702  146227 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22279 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:4e:00.0, compute capability: 8.9


Model: "TransformerTTS"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder (Encoder)               │ ?                      │    70,274,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ tokpos                     │ ?                      │    65,536,256 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dropout (Dropout)          │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_0 (EncoderLayer)     │ ?                      │       789,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_1 (EncoderLayer)     │ ?                      │       789,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_2 (EncoderLayer)     │ ?                      │       789,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_3 (EncoderLayer)     │ ?                      │       789,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_4 (EncoderLayer)     │ ?                      │       789,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_5 (EncoderLayer)     │ ?                      │       789,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (DecoderTTS)            │ ?                      │     6,472,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ melpos                     │ ?                      │       152,320 │
│ (MelPositionalProjection)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ dropout (Dropout)          │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_0 (DecoderLayer)     │ ?                      │     1,053,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_1 (DecoderLayer)     │ ?                      │     1,053,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_2 (DecoderLayer)     │ ?                      │     1,053,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_3 (DecoderLayer)     │ ?                      │     1,053,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_4 (DecoderLayer)     │ ?                      │     1,053,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│    └ block_5 (DecoderLayer)     │ ?                      │     1,053,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mel_head (Dense)                │ (None, 2000, 80)       │        20,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ stop_head (Dense)               │ (None, 2000, 1)        │           257 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ postnet (PostNet)               │ ?                      │     4,352,400 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 81,121,073 (309.45 MB)

 Trainable params: 81,116,817 (309.44 MB)

 Non-trainable params: 4,256 (16.62 KB)

✅ Weights loaded.


In [4]:
# ---------- کمکـی‌ها برای برگرداندن mel_norm[-1,1] به موج ----------
def db_to_power(db): return tf.pow(10.0, db / 10.0)

def denorm_mel(mel_norm):
    mel01  = (mel_norm + 1.0) * 0.5        # [-1,1] -> [0,1]
    mel_db = mel01 * 100.0 - 100.0         # -> [-100, 0] dB
    return db_to_power(mel_db)             # -> power

def mel_to_linear_power(mel_power, sr=SR, n_fft=N_FFT, n_mels=N_MELS, fmin=FMIN, fmax=FMAX):
    mel_mat = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=n_mels,
        num_spectrogram_bins=n_fft // 2 + 1,
        sample_rate=float(sr),
        lower_edge_hertz=float(fmin),
        upper_edge_hertz=float(fmax),
        dtype=tf.float32
    )  # (n_fft//2+1, n_mels)
    mel_pinv = tf.linalg.pinv(mel_mat)               # (n_mels, n_fft//2+1)
    linear_power = tf.matmul(mel_power, mel_pinv)    # (T, n_fft//2+1)
    return tf.maximum(linear_power, 1e-10)

def griffin_lim_from_mag(mag, n_fft=N_FFT, hop=HOP, win=WIN, n_iter=60):
    mag = tf.cast(mag, tf.float32)

    # --- init random phase (بدون 1j) ---
    phases = tf.random.uniform(tf.shape(mag), 0.0, 2.0*np.pi, dtype=tf.float32)
    angles = tf.complex(tf.cos(phases), tf.sin(phases))            # (T, n_fft//2+1) complex64
    S = tf.cast(mag, tf.complex64) * angles

    def _stft(w):
        return tf.signal.stft(w, frame_length=win, frame_step=hop,
                              fft_length=n_fft, window_fn=tf.signal.hann_window)
    def _istft(S_):
        return tf.signal.inverse_stft(S_, frame_length=win, frame_step=hop,
                                      window_fn=tf.signal.hann_window)

    for _ in range(n_iter):
        wav = _istft(S)
        S_est = _stft(wav)
        ang = tf.math.angle(S_est)                                 # float32
        phase = tf.complex(tf.cos(ang), tf.sin(ang))               # complex64
        S = tf.cast(mag, tf.complex64) * phase

    wav = _istft(S)
    wav = wav / (tf.reduce_max(tf.abs(wav)) + 1e-6)                # [-1,1]
    return tf.squeeze(wav)


def mel_to_wav_griffinlim(mel_norm, n_iter=60):
    mel_power = denorm_mel(mel_norm)
    linear_power = mel_to_linear_power(mel_power)
    mag = tf.sqrt(tf.maximum(linear_power, 1e-10))
    return griffin_lim_from_mag(mag, n_iter=n_iter)

# ---------- مدل و توکنایزر ----------
# فرض: core همون TransformerTTS شماست که build شده و وزن‌هاش لود شدن
# core.load_weights("checkpoints/tts_core_best.weights.h5")

tok = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", use_fast=False, src_lang="eng_Latn")

def tokenize_texts(texts, tok, max_len=256, pad_id=1):
    ids = [tok.encode(t, add_special_tokens=True, src_lang="eng_Latn")[:max_len] for t in texts]
    lens = [len(x) for x in ids]
    ids_pad = tf.keras.preprocessing.sequence.pad_sequences(ids, maxlen=max_len, padding="post", value=pad_id)
    return tf.constant(ids_pad, tf.int32), tf.constant(lens, tf.int32)



# (اختیاری) ذخیره‌ی WAV روی دیسک
# tf.io.write_file("tts_out.wav", tf.audio.encode_wav(tf.expand_dims(wav, -1), sample_rate=SR))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# ---------- اینفرنس ----------
texts = ["Please use our dedicated channels for questions and discussion"]
enc_ids, _ = tokenize_texts(texts, tok, max_len=256, pad_id=1)

# greedy_generate شما خروجی PostNet را برمی‌گرداند (mel_hat)
mel_hat, stop_probs = core.greedy_generate_fast(
    enc_ids,
    max_steps=600,      # برای تست، بعداً زیاد کن
    min_steps=40,
    stop_threshold=0.55,
    window=6,
    patience=2,
    check_stop_every=10,
    verbose=True
)
mel_one = mel_hat[0].numpy()
wav = mel_to_wav_griffinlim(mel_one, n_iter=32)  # قبلاً 64 یا 60 بود
Audio(wav.numpy(), rate=SR)

/home/dev371/.local/lib/python3.10/site-packages/keras/src/ops/nn.py:944: UserWarning: You are using a softmax over axis 3 of a tensor of shape (1, 8, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


gen step 50 stop~ 6.85497423e-11
gen step 100 stop~ 2.43097036e-11
gen step 150 stop~ 1.47264326e-10
gen step 200 stop~ 8.39547876e-11
gen step 250 stop~ 1.84332841e-10
gen step 300 stop~ 3.44713911e-08
gen step 350 stop~ 1.53705448e-09
gen step 400 stop~ 1.09547399e-10
gen step 450 stop~ 5.28806421e-09
gen step 500 stop~ 4.72814177e-10
gen step 550 stop~ 3.95726757e-10
gen step 600 stop~ 3.29723887e-10


I0000 00:00:1761153757.348738  146227 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1761153758.664666  146227 cuda_solvers.cc:175] Creating GpuSolver handles for stream 0x5ac09280d7a0
